In [ ]:
Установка зависимостей (только если вы запускаете это отдельно от обучения)
!git clone https://github.com/rosinality/stylegan2-pytorch.git
%cd stylegan2-pytorch
!pip install Ninja
!pip install git+https://github.com/openai/CLIP.git -q
!pip install dlib
!gdown http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O dlib_models/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk dlib_models/shape_predictor_68_face_landmarks.dat.bz2

# Возвращаемся в корневую папку, если вы в stylegan2-pytorch
%cd ..

# Создание структуры папок, если она еще не существует
%mkdir -p pretrained_models
%mkdir -p data/inversion
%mkdir -p dlib_models

# Импорты
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from argparse import Namespace
from torchvision import transforms
import dlib
import copy

# Импортируем наши собственные модули
from modules.model import Generator # Предполагаем, что Generator также содержит pSp или pSp определен в отдельном файле, который вы импортируете
from modules.utils import align_face, run_on_batch # Ваши вспомогательные функции

# Загрузите класс pSp из вашего проекта encoder4editing.
# Вставьте сюда определение класса pSp, если он не находится в `modules/model.py`
# или убедитесь, что `modules/model.py` его содержит.
# Пример (вставьте из вашего кода encoder4editing):
# class pSp(torch.nn.Module):
#     def __init__(self, opts):
#         super(pSp, self).__init__()
#         self.encoder = Encoder(50, 512) # Placeholder, replace with actual Encoder init
#         self.decoder = Generator(opts.stylegan_size, opts.latent_dim, opts.n_mlp, channel_multiplier=opts.channel_multiplier)
#         self.opts = opts
#     def forward(self, x, resize=True, latent_mask=None, input_code=None, randomize_noise=True, return_latents=False, alpha=None):
#         codes = self.encoder(x)
#         if input_code is not None:
#             codes = input_code + codes
#         if latent_mask is None:
#             latent_mask = [True] * self.opts.n_styles
#         # ... (rest of pSp forward pass) ...
#         return images, codes

# Если pSp находится в modules/model.py, импортируйте его:
# from modules.model import pSp # РАССКОММЕНТИРУЙТЕ, если pSp определен в modules/model.py

# Зададим девайс
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используется устройство: {device}")

In [ ]:
# Загрузка StyleGAN-NADA Generator (ваша обученная модель)
size = 1024
latent_dim = 512
n_mlp = 8
channel_multiplier = 2

# Инициализируем генератор для загрузки обученных весов
custom_generator = Generator(size, latent_dim, n_mlp, channel_multiplier=channel_multiplier).to(device)
custom_generator.eval() # Переводим в режим инференса

# Укажите путь к вашей обученной модели
trained_model_path = "pretrained_models/my_custom_stylegan_generator.pth"
if os.path.exists(trained_model_path):
    custom_generator.load_state_dict(torch.load(trained_model_path, map_location=device))
    print(f"Обученная модель StyleGAN-NADA загружена из {trained_model_path}")
else:
    print(f"Ошибка: Обученная модель по пути {trained_model_path} не найдена. Убедитесь, что она была сохранена после обучения.")
    # Fallback to the original generator if trained model not found (for demonstration)
    # This might not produce styled images if the original generator wasn't styled.
    checkpoint = torch.load('pretrained_models/stylegan2-ffhq-config-f.pt', map_location=device)
    custom_generator.load_state_dict(checkpoint["g_ema"])
    print("Загружена исходная StyleGAN2 FFHQ модель.")


# Define image transformations for pSp
transform = transforms.Compose([
    transforms.Resize((256, 256)), # pSp обычно принимает 256x256
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) # Нормализация в [-1, 1]
])

In [ ]:
! git clone https://github.com/omertov/encoder4editing.git
%cd encoder4editing
# Импортируем необходимые библиотеки
from models.psp import pSp  # Импортируем модель pSp
from argparse import Namespace

# Load pre-trained pSp model
# Убедитесь, что opts и net определены
# Вам может потребоваться загрузить e4e_ffhq_encode.pt вручную, если его нет
# !gdown --id 1_V7g3Q38M-M-0s9W-t5gT_Wb-29N0J9V -O encoder4editing/e4e_ffhq_encode.pt # Пример ссылки

model_path = "/content/encoder4editing/e4e_ffhq_encode.pt" # Убедитесь, что этот путь верен
# Если `pSp` класс не в вашем `modules`, вам, возможно, придется импортировать его из оригинального репозитория `encoder4editing`
# Или скопировать его определение в `modules/model.py`
# Для этого примера я предполагаю, что pSp класс определен и доступен.
# Вот как загрузить pSp и его опции (вам может потребоваться настроить пути):
try:
    ckpt_psp = torch.load(model_path, map_location="cpu")
    opts_dict = ckpt_psp["opts"]
    opts_dict["checkpoint_path"] = model_path
    opts = Namespace(**opts_dict)

    # Assuming pSp class is available globally or imported
    # This is a critical point: Ensure your pSp class is defined or imported correctly.
    # If pSp is a sub-module of the original StyleGAN2-pytorch, then `from model import Generator` might suffice
    # IF pSp is implemented within Generator or an accessible part.
    # Otherwise, you need to import/define the pSp architecture.

    # Placeholder for pSp model initialization
    # If your pSp model is just a custom `Generator` with an encoder part, adjust accordingly.
    # Example for actual pSp model from encoder4editing:
    # from models.psp import pSp  # If you have pSp in a separate models/psp.py
    # net_psp = pSp(opts).to(device)
    # net_psp.eval()
    # net_psp.load_state_dict(ckpt_psp['state_dict'], strict=False)

    # For simplicity, if pSp IS the Generator and its encoder, you might just use it:
    # For now, let's assume `pSp` needs to be imported or its class definition provided.
    # If not using the actual `pSp` encoder, you'd need a different way to get `latent`.

    print("Модель pSp (или аналогичный энкодер) успешно загружена!")
except FileNotFoundError:
    print(f"Ошибка: Модель pSp по пути {model_path} не найдена. Пожалуйста, убедитесь, что она скачана.")
    net_psp = None # Set to None to prevent errors later

net = pSp(opts)
net.eval()
net.to(device)
print("Model successfully loaded!")

# Загрузка модели dlib для выравнивания лица
landmark_model_path = "dlib_models/shape_predictor_68_face_landmarks.dat"
if not os.path.exists(landmark_model_path):
    print(f"Скачивание модели Dlib для выравнивания лица...")
    !wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O dlib_models/shape_predictor_68_face_landmarks.dat.bz2
    !bzip2 -dk dlib_models/shape_predictor_68_face_landmarks.dat.bz2
    print("Модель Dlib скачана.")

predictor = dlib.shape_predictor(landmark_model_path)


# Директория с изображениями для инверсии
images_to_invert_dir_path = "data/inversion" #@ param {"type": "string"}
os.makedirs(images_to_invert_dir_path, exist_ok=True)

# Загрузите тестовые изображения в data/inversion/ (например, с помощью Drag-and-Drop в Colab)
# Пример:
# !wget -O data/inversion/test_face.jpg https://upload.wikimedia.org/wikipedia/commons/b/b2/Damon_Runyon.jpg

# Процесс инверсии и стилизации
if net_psp: # Только если pSp модель успешно загружена
    for image_name in os.listdir(images_to_invert_dir_path):
        if not image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue # Пропускаем не-изображения

        image_path = os.path.join(images_to_invert_dir_path, image_name)

        original_image = Image.open(image_path).convert("RGB")

        # Выравнивание лица
        input_image = align_face(filepath=image_path, predictor=predictor)
        input_image = input_image.resize((256, 256))
        transformed_image = transform(input_image)

        with torch.no_grad():
            # Получаем латентный код W+ от pSp модели
            # Здесь `net_psp` должен быть вашей pSp моделью.
            # Если `pSp` является классом `Generator` (или его частью), то это:
            # _, latent = custom_generator.style(transformed_image.unsqueeze(0)) # Это неверно для pSp. pSp имеет свой Forward.

            # Предполагаем, что `net_psp` - это ваш загруженный pSp энкодер
            result_image_from_psp, latent = run_on_batch(transformed_image.unsqueeze(0), net_psp, device)

        # Генерируем стилизованное изображение с помощью вашей обученной StyleGAN-NADA
        # `latent` здесь уже должен быть в пространстве W+ (1, 18, 512)
        custom_generator.eval() # Убедимся, что генератор в режиме eval
        styled_image_final, _ = custom_generator([latent], input_is_latent=True, randomize_noise=False)

        # Plot results
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))

        axs[0].imshow(np.asarray(original_image))
        axs[0].set_title("Original Image")
        axs[0].axis("off")

        # Scale from [-1, 1] to [0, 1] for display
        styled_image_np = (styled_image_final[0].detach().cpu().clamp(-1, 1) + 1) / 2
        styled_image_np = styled_image_np.permute(1, 2, 0).numpy()

        axs[1].imshow(styled_image_np)
        axs[1].set_title(f"Styled (from {image_name})")
        axs[1].axis("off")

        plt.suptitle(f"Inference for: {image_name}")
        plt.show()
else:
    print("Невозможно выполнить инференс: Модель pSp не загружена.")